<a href="https://colab.research.google.com/github/xyzesther/decoding_digital_personas/blob/main/MBTI_Prediction_with_Social_Media_Content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preprocessing

### Load and inspect data

In [ ]:
import kagglehub
import os
import pandas as pd

# Download latest version
dir = kagglehub.dataset_download("datasnaek/mbti-type")

# Find the CSV file within the downloaded directory
for filename in os.listdir(dir):
    if filename.endswith(".csv"):
        path = os.path.join(dir, filename)
        break

print("Path to dataset files:", path)

# Load the dataset
df = pd.read_csv(path)

print(f"Dataset shape: {df.shape}\n")

df.head()

# Check for missing values
print("Missing values:\n", df.isnull().sum())

# Display unique MBTI types
print("Unique MBTI Types:\n", df['type'].unique())

100%|██████████| 24.4M/24.4M [00:01<00:00, 13.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/datasnaek/mbti-type/versions/1/mbti_1.csv
Dataset shape: (8675, 2)

Missing values:
 type     0
posts    0
dtype: int64
Unique MBTI Types:
 ['INFJ' 'ENTP' 'INTP' 'INTJ' 'ENTJ' 'ENFJ' 'INFP' 'ENFP' 'ISFP' 'ISTP'
 'ISFJ' 'ISTJ' 'ESTP' 'ESFP' 'ESTJ' 'ESFJ']


In [ ]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
import re
import demoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert MBTI labels into four binary features (E/I, S/N, T/F, J/P)
def mbti_to_binary(mbti_type):
    return [
        1 if mbti_type[0] == 'E' else 0,
        1 if mbti_type[1] == 'S' else 0,
        1 if mbti_type[2] == 'T' else 0,
        1 if mbti_type[3] == 'J' else 0
    ]

df[['E/I', 'S/N', 'T/F', 'J/P']] = df['type'].apply(lambda x: pd.Series(mbti_to_binary(x)))

df.head()


,type,posts,E/I,S/N,T/F,J/P
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,0,0,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,1,0,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,0,0,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",0,0,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,1,0,1,1


In [ ]:
# Text preprocessing
demoji.download_codes()

def preprocess_text(text):
    # Convert emojis to text
    text = demoji.replace_with_desc(text, sep=" ")

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Handle special characters and case
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    # Handle post separators (|||)
    text = text.replace('|||', ' ')

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df['clean_text'] = df['posts'].apply(preprocess_text)

<ipython-input-5-c4b6a6d0eebe>:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


KeyboardInterrupt: 